## Analysis for paper writing
Author: Sahar H. El Abbadi
Date: 2023-04-12


## Summarize distribution of overpasses

Total number of overpasses, missing data, max and min given to each team, number that passed Stanford QC

Missing overpasses: documented as an overpass by Stanford, but not included in Operator Report file

#### Carbon Mapper overpass summary for results

In [ ]:
from writing_analysis import operator_releases_summary_stats
# Carbon Mapper
operator = 'Carbon Mapper'
operator_releases_summary_stats(operator)

### GHGSat summary for results

In [ ]:
from writing_analysis import operator_releases_summary_stats
# GHGSat
operator = 'GHGSat'
operator_releases_summary_stats(operator)

In [ ]:
# Check GHGSat's internal QC for points they quantified but flagged as sub-optimal

from methods_source import load_operator_report_dictionary, load_overpass_summary
operator = 'GHGSat'
ghg_report = load_operator_report_dictionary()['ghg_1']

# QC flag of GH-2 means that emissions were quantified despite sub-optimal conditions.
# QC flag of GH-4 means diffuse emissions visible over site

ghg_report_quantified = ghg_report.loc[ghg_report.QuantifiedPlume == True]
qc_mask = (ghg_report_quantified['QCFlag'] == 'GH-2') | (ghg_report_quantified['QCFlag'] == 'GH-4')
ghg_report_poor_conditions = ghg_report_quantified.loc[qc_mask]
# list of overpass ID's with poor conditions but quantified
poor_condition_overpasses = ghg_report_poor_conditions['overpass_id']
overpasses = load_overpass_summary(operator, stage=1)

# Find the overpasses in the overpass summary, to see which ones of them pass Stanford QC
poor_condition_overpass_summary = overpasses[overpasses['overpass_id'].isin(poor_condition_overpasses)]

# Select overpasses that pass Stanford QC
poor_condition_overpasses_pass_SU = poor_condition_overpass_summary.loc[poor_condition_overpass_summary.stanford_kept == True]

print(f'Number of overpasses that pass SU quality control but are flagged by GHGSat as sub-optimal conditions for quantification: {len(poor_condition_overpasses_pass_SU)}')

In [ ]:
# Further investigate GHGSat false negatives, as they detect small releases but also had some higher false negatives
from methods_source import load_overpass_summary, classify_confusion_categories

from writing_analysis import print_overpass_info

operator = 'GHGSat'
overpasses = load_overpass_summary(operator, stage=1)
pass_su_qc = overpasses.loc[overpasses.qc_summary == 'pass_all']
true_positives, false_positives, true_negatives, false_negatives = classify_confusion_categories(pass_su_qc)

# How many of these releases were less than 5 kgh?
small_false_negatives = false_negatives.loc[false_negatives['release_rate_kgh'] < 5]
print(f'Number of false negatives that were less than 5 kgh: {len(small_false_negatives)}\n')

# How many of these releases were less than 5 kgh?
large_false_negatives = false_negatives.loc[false_negatives['release_rate_kgh'] > 5]
print(f'False negatives that were above 5 kgh:\n')
print(large_false_negatives['release_rate_kgh'])
print(f'\n')

# print details of each of hte false negatives that is over 5 kgh
for index, row in large_false_negatives.iterrows():
    print_overpass_info(row)

### Kairos summary for results

In [ ]:
from writing_analysis import operator_releases_summary_stats
# GHGSat
operator = 'Kairos'
operator_releases_summary_stats(operator)

In [ ]:
# Additional Kairos analysis
from methods_source import load_overpass_summary
# How many overpasses were at or below 10 kg/hr
operator = 'Kairos'
overpasses = load_overpass_summary(operator, stage=1)
pass_all_qc = overpasses.loc[overpasses.pass_all_qc == True]

lower_end = pass_all_qc.loc[pass_all_qc.release_rate_kgh < 15]
print(f'Total number of releases for {operator} less than 15 kgh: {len(lower_end)} ')

### MethaneAIR summary for results


In [ ]:
### MethaneAIR
from writing_analysis import operator_releases_summary_stats
operator = 'MethaneAIR'
operator_releases_summary_stats(operator)

### Scientific Aviation summary for results


In [ ]:
### SciAV
from writing_analysis import operator_releases_summary_stats
operator = 'Scientific Aviation'
operator_releases_summary_stats(operator, strict_discard=True)

In [ ]:
# Additional Scientific Aviation analysis
from methods_source import load_overpass_summary
# Which dates contained SCIAV qc'ed points
operator = 'Scientific Aviation'
overpasses = load_overpass_summary(operator, strict_discard=True, stage=1)

relevant_col = 'overpass_datetime'
fail_sciav_qc_dates = overpasses.loc[overpasses.operator_kept == False][relevant_col]
print(fail_sciav_qc_dates)

## Analyze Error Bar Profile

What fraction of quantification estimates have upper or lower bounds that cross the parity line? Functions for testing parity apply the following filters to the operator overpasses:
  - Must pass all QC
  - Operator quantification is not null to not include NAN values as False (for not crossing parity line)

In [2]:
from writing_analysis import test_parity_all_stages

# Carbon Mapper
operator = 'Carbon Mapper'
strict_discard = False
test_parity_all_stages(operator)

# GHGSat
operator = 'GHGSat'
strict_discard = False
test_parity_all_stages(operator)

# MethaneAIR
operator = 'MethaneAIR'
strict_discard = False
test_parity_all_stages(operator)

# Scientific Aviation
operator = 'Scientific Aviation'
strict_discard = True
test_parity_all_stages(operator)

Fraction of Carbon Mapper Stage 1 overpasses with 95% CI that encompasses parity line: 89%
Fraction of Carbon Mapper Stage 2 overpasses with 95% CI that encompasses parity line: 76%
Fraction of Carbon Mapper Stage 3 overpasses with 95% CI that encompasses parity line: 80%


Fraction of GHGSat Stage 1 overpasses with 95% CI that encompasses parity line: 93%
Fraction of GHGSat Stage 2 overpasses with 95% CI that encompasses parity line: 84%
Fraction of GHGSat Stage 3 overpasses with 95% CI that encompasses parity line: 84%


Fraction of MethaneAIR Stage 1 overpasses with 95% CI that encompasses parity line: 83%


Fraction of Scientific Aviation Stage 1 overpasses with 95% CI that encompasses parity line: 67%




In [3]:
# Closer look at GHGSat parity plot points and how many cross the parity line

from writing_analysis import calc_parity_intersection
import pathlib
import pandas as pd
operator = 'GHGSat'

ghg_1_df = calc_parity_intersection(operator, 1, strict_discard=False)
ghg_1_df.to_csv(pathlib.PurePath('03_results', 'paper_analysis', 'ghg_1_df_parity_compare.csv'))

ghg_2_df = calc_parity_intersection(operator, 2, strict_discard=False)
ghg_2_df.to_csv(pathlib.PurePath('03_results', 'paper_analysis', 'ghg_2_df_parity_compare.csv'))

compare_ghg = pd.DataFrame()
compare_ghg['stage1_95CI'] = ghg_1_df['operator_95CI_bounds'] # how big are stage 1 error bars?
compare_ghg['stage2_95CI'] = ghg_2_df['operator_95CI_bounds'] # how big are stage 2 error bars?

compare_ghg['CI_ratio'] = compare_ghg['stage2_95CI'] / compare_ghg['stage1_95CI']
compare_ghg['stage1_cross_parity'] = ghg_1_df['intersect_parity_line'] # cross parity in stage 1?
compare_ghg['stage2_cross_parity'] = ghg_2_df['intersect_parity_line'] # cross parity in stage 2?

stage1_95CI_mean = compare_ghg.stage1_95CI.mean()
stage2_95CI_mean = compare_ghg.stage2_95CI.mean()
mean_ratio = compare_ghg.CI_ratio.mean()
stage1_cross_parity = len(compare_ghg.loc[compare_ghg.stage1_cross_parity == True])
stage2_cross_parity = len(compare_ghg.loc[compare_ghg.stage2_cross_parity == True])
print(f'Mean 95% CI error bar length for Stage 1: {stage1_95CI_mean:0.2f}')
print(f'Mean 95% CI error bar length for Stage 2: {stage2_95CI_mean:0.2f}')
print(f'Average ratio between Stage 2 and Stage 1: {mean_ratio:0.4f}')
print(f'Minimum CI ratio: {compare_ghg.CI_ratio.min():0.2f}')
print(f'Minimum CI ratio: {compare_ghg.CI_ratio.max():0.2f}\n')


# flip from cross parity to not cross parity
mask_true_to_false = (compare_ghg.stage1_cross_parity == True) & (compare_ghg.stage2_cross_parity == False)
flip_T_to_F = compare_ghg.loc[mask_true_to_false]

print(f'Number of points that switch from crossing parity line in Stage 1 to not crossing parity line in Stage 2: {len(flip_T_to_F)}\n')

# flip from cross parity to not cross parity
mask_false_to_true = (compare_ghg.stage1_cross_parity == False) & (compare_ghg.stage2_cross_parity == True)
flip_F_to_T = compare_ghg.loc[mask_false_to_true]

print(f'Number of points that switch from NOT crossing parity line in Stage 1 to crossing parity line in Stage 2: {len(flip_F_to_T)}\n')


Fraction of GHGSat Stage 1 overpasses with 95% CI that encompasses parity line: 93%
Fraction of GHGSat Stage 2 overpasses with 95% CI that encompasses parity line: 84%
Mean 95% CI error bar length for Stage 1: 113.16
Mean 95% CI error bar length for Stage 2: 71.71
Average ratio between Stage 2 and Stage 1: 0.6118
Minimum CI ratio: 0.10
Minimum CI ratio: 1.08

Number of points that switch from crossing parity line in Stage 1 to not crossing parity line in Stage 2: 16

Number of points that switch from NOT crossing parity line in Stage 1 to crossing parity line in Stage 2: 5



In [5]:
# Scientific Aviation
operator = 'Scientific Aviation'
strict_discard = True
data = calc_parity_intersection(operator, stage=1, strict_discard=True)
cross_parity = data.loc[data['intersect_parity_line'] == True]
print(f'Number of points that cross parity line: {len(cross_parity)}')

Fraction of Scientific Aviation Stage 1 overpasses with 95% CI that encompasses parity line: 62%
Number of points that cross parity line: 5


### Kairos distance from parity line

Kairos does not report uncertainty for their quantification estimates. Thus, we cannot look at how many of the points have error bars that cross the parity line.

Instead, we look at how many are within 25% and 50% of the true value.

In [17]:
from methods_source import load_overpass_summary
import pandas as pd

kairos_1 = load_overpass_summary('Kairos', 1, strict_discard=False)

# Filter for pass all QC
kairos_1 = kairos_1.loc[kairos_1['qc_summary']=='pass_all']

# Remove zero releases:
kairos_1 = kairos_1.loc[kairos_1['non_zero_release'] == True]

# Remove non-detected releases (operator quantification > 0)
kairos_1 = kairos_1.loc[kairos_1['operator_quantification'] > 0]

# Evan in Sherwin et al 2023 says 86% of Kairos quantification estimates fall within +/- 50% of the metered value

kairos_accuracy = pd.DataFrame()
kairos_accuracy['release_rate_kgh'] = kairos_1['release_rate_kgh']
kairos_accuracy['operator_quantification'] = kairos_1['operator_quantification']
kairos_accuracy['release_rate_0.50x'] = kairos_accuracy['release_rate_kgh'] * 0.5
kairos_accuracy['release_rate_1.50x'] = kairos_accuracy['release_rate_kgh'] * 1.5
kairos_accuracy['release_rate_0.25x'] = kairos_accuracy['release_rate_kgh'] * 0.75
kairos_accuracy['release_rate_1.25x'] = kairos_accuracy['release_rate_kgh'] * 1.25

multipliers = ['.25', '.50']
accuracy_record = []
for index, row in kairos_accuracy.iterrows():
    new_row = {}
    for multipler in multipliers:

        upper_col = f'release_rate_1{multipler}x'
        lower_col = f'release_rate_0{multipler}x'
        upper_bound = row[upper_col]
        lower_bound = row[lower_col]
        estimate = row['operator_quantification']
        # print(f'estimate: {estimate}')
        # print(f'upper_bound: {upper_bound}')
        # print(f'low_bound: {lower_bound}')
        if (estimate <= upper_bound) and (estimate >= lower_bound):
            record = True
        else:
            record = False
        new_row['index'] = index
        new_row[f'within_{multipler[1:3]}%'] = record
    accuracy_record.append(new_row)

test_accuracy = pd.DataFrame.from_records(accuracy_record, index='index')
# add_cols=add_cols.astype({"within_25%":bool,"within_50%":bool})

print(f'Sanity check:')
test_mask = (test_accuracy['within_25%'] == True) & (test_accuracy['within_50%'] == False)
print(f'Number of points within 50% accuracy but not within 25% accuracy: {len(test_accuracy.loc[test_mask])}\n')

total_points = len(test_accuracy)
test_25_percent = len(test_accuracy[test_accuracy['within_25%'] == True])
print(f'Number of points within 25% of metered release rate: {test_25_percent}')
percent_within_25_percent = test_25_percent / total_points * 100
print(f'Percentage of measurements within 25% of metered rate: {percent_within_25_percent:.1f}%')

test_50_percent = len(test_accuracy[test_accuracy['within_50%'] == True])
print(f'Number of points within 25% of metered release rate: {test_50_percent}')
percent_within_50_percent = test_50_percent / total_points * 100
print(f'Percentage of measurements within 25% of metered rate: {percent_within_50_percent:.1f}%')

Sanity check:
Number of points within 50% accuracy but not within 25% accuracy: 0

Number of points within 25% of metered release rate: 47
Percentage of measurements within 25% of metered rate: 37.9%
Number of points within 25% of metered release rate: 91
Percentage of measurements within 25% of metered rate: 73.4%


## Analyze Error Profile - Absolute, Percent Error

- Examine percent and absolute error profile in quantification estimates
- Only include points that pass all QC
- Residuals compare the quantification estimate to the OLS best fit


First, determine the ranges for residuals, absolute error, and percent error. These are used in making the figures in 11_paper_figures

In [ ]:
from writing_analysis import determine_relevant_error_ranges
# Determine what to use for axis across all plots
# First find min and max for residuals
from methods_source import abbreviate_op_name
import pandas as pd
from writing_analysis import summarize_operator_stages_defaults


operator_stages = summarize_operator_stages_defaults()

summary = []
for index, test in operator_stages.iterrows():
    operator = test['operator']
    stage = test['stage']
    strict_discard = test['strict_discard']
    plot_range = determine_relevant_error_ranges(operator, stage, 'pass_all', strict_discard)
    summary.append(plot_range)

range_summary = pd.DataFrame(summary)
residual_ylim = [range_summary.min_residual.min(), range_summary.max_residual.max()]
percent_error_ylim = [range_summary.min_error_percent.min(), range_summary.max_error_percent.max()]
absolute_error_ylim = [range_summary.min_error_absolute.min(), range_summary.max_error_absolute.max()]

print(f'Residual range is {residual_ylim[0]:.0f} to {residual_ylim[1]:.0f} kgh')
print(f'Percent error range is {percent_error_ylim[0]:.0f}% to {percent_error_ylim[1]:.0f}%')
print(f'Absolute error range is {absolute_error_ylim[0]:.0f} to {absolute_error_ylim[1]:.0f}')

In [ ]:
# Examine the error profile for each operator
from writing_analysis import calculate_residuals_and_error
operator = 'Carbon Mapper'
stage = 1
op_stage = calculate_residuals_and_error(operator, stage, qc_status='pass_all', strict_discard=False, time_ave=60, gas_comp_source='km')


In [7]:
# Examine the error profile for each operator
from writing_analysis import calculate_residuals_and_error
operator = 'Scientific Aviation'
stage = 1
op_stage = calculate_residuals_and_error(operator, stage, qc_status='pass_all', strict_discard=True, time_ave=60, gas_comp_source='km')
print(op_stage)

    overpass_id   overpass_datetime  zero_release  non_zero_release  \
5             6 2022-11-10 18:46:34         False              True   
6             7 2022-11-10 19:26:04         False              True   
7             8 2022-11-10 20:04:59          True             False   
8             9 2022-11-10 20:47:38         False              True   
9            10 2022-11-10 21:25:50         False              True   
11           12 2022-11-11 19:14:36         False              True   
12           13 2022-11-11 19:49:30         False              True   
14           15 2022-11-11 20:56:01         False              True   
15           16 2022-11-11 21:30:40          True             False   
16           17 2022-11-11 22:42:59         False              True   

    operator_kept  stanford_kept  phase_iii  pass_all_qc  altitude_feet  \
5            True           True          0         True            NaN   
6            True           True          0         True            